In [ ]:
#AnyBoard! use a blue or orange expo marker as your trace head for best results

import cv2 as cv
import numpy as np
import time

cap=cv.VideoCapture(0) #0 means webcam
width= int(cap.get(3))#3 and 4 are hardcoded properties
height=int(cap.get(4))
erase=False

print(width)
print(height)

points=[] #circle holder
segments=[]

points_orange=[] #circle holder
segments_orange=[]

if cap.isOpened():
    print("open")

def empty(a):
    pass
def getBlueColorMask(img):

    hsv=cv.cvtColor(img,cv.COLOR_BGR2HSV)
    lower=np.array([100,90,50])#bgr 
    upper=np.array([130,255,255])#bgr
    return cv.inRange(hsv,lower,upper)  

def getOrangeColorMask(img):
    
    hsv=cv.cvtColor(img,cv.COLOR_BGR2HSV)
    lower=np.array([0,150,180])#bgr
    upper=np.array([30,255,255])#bgr
    return cv.inRange(hsv,lower,upper)  

def getContours(img,contour_img):
    x=0
    y=0
    global points,segments
    blueDetected=False
    contours,hierarchy=cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)
    for c in contours:
        area = cv.contourArea(c)
        if area<3000:
            peri=cv.arcLength(c,True)
            approx=cv.approxPolyDP(c,0.01*peri,True)
            x,y,w,h=cv.boundingRect(approx) 
            #cv.rectangle(contour_img,(x,y),(x+w,y+h),(255,0,0),5)#frame,top left, bottom right,color, fill (coordinates input as tuples)
            blueDetected=True              
    if( x<100 & y<100):
        segments=[]
    if(blueDetected):
        points.append((x,y))
        i=0
        for c in points:
            if(len(points)>0):
                if(c!=points[0]):
                    cv.line(contour_img, points[i-1],points[i],(0,0,255),6)
            i+=1
    else:
        segments.append(points)
        points=[]

def getOrangeContours(img,contour_img):
    x=0
    y=0
    global points_orange,segments_orange
    orangeDetected=False
    contours,hierarchy=cv.findContours(img,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_NONE)
    for c in contours:
        area = cv.contourArea(c)
        if area<3000:
            peri=cv.arcLength(c,True)
            approx=cv.approxPolyDP(c,0.01*peri,True)
            x,y,w,h=cv.boundingRect(approx) 
            # cv.rectangle(contour_img,(x,y),(x+w,y+h),(255,0,0),5)#frame,top left, bottom right,color, fill (coordinates input as tuples)
            orangeDetected=True              
    if( x<100 & y<100):
        segments_orange=[]
    if(orangeDetected):
        points_orange.append((x,y))
        print(points_orange)
        i=0
        for c in points_orange:
            print("inside trace loop")
            if(len(points_orange)>0):
                if(c!=points_orange[0]):
                    cv.line(contour_img, points_orange[i-1],points_orange[i],(0,0,255),6)
            i+=1
    else:
        segments_orange.append(points_orange)
        points_orange=[]

def drawSegments():
    global points
    global segments
    print(segments)
    thing = (1,1)
    i=0
    for s in segments:
        i=0
        for p in s:
            if(len(s)>=2):
                if(p!=thing and s[i]!=s[0]):
                    print ("inside loop")

                    cv.line(contour_img, s[i-1],s[i]  ,(255,0,0),6)
                    thing = s[i]
            i+=1
def drawOrangeSegments():
    global points_orange
    global segments_orange
    print("drawing orange")
    thing = (1,1)
    x=0
    for s in segments_orange:
        x=0
        print(x)
        for p in s:
            if(len(s)>=2):
                if(p!=thing and s[x]!=s[0]):
                    cv.line(contour_img, s[x-1],s[x]  ,(0,165,255),6)
                    thing = s[x]
            x+=1

cv.namedWindow('color values',cv.WINDOW_NORMAL)
cv.createTrackbar('blue lightness','color values',50,255,empty)
cv.createTrackbar('orange lightness','color values',170,255,empty)
cv.createTrackbar('orange saturation','color values',90,255,empty)


while(True):
    ret, frame = cap.read()
    contour_img=frame.copy()
    blur_img=cv.GaussianBlur(frame,(9,9),1)
    kernel=np.ones((3,3))

    mask_orange_img =getOrangeColorMask(blur_img)
    erode_img_orange=cv.erode(mask_orange_img,kernel,iterations=3)
    dilate_img_orange=cv.dilate(erode_img_orange,kernel,iterations=3)
    getOrangeContours(dilate_img_orange,contour_img)
    drawOrangeSegments()

    mask_img = getBlueColorMask(blur_img)
    erode_img=cv.erode(mask_img,(5,5),iterations=3)
    dilate_img=cv.dilate(erode_img,kernel,iterations=3)
    getContours(dilate_img,contour_img)
    drawSegments()

    cv.rectangle(contour_img, (0,0),(80,480),(0,255,0),3)
    font = cv.FONT_HERSHEY_SIMPLEX
    contour_img = cv.flip(contour_img,1)
    cv.putText(contour_img,'erase',(width-85,50),font,1,(0,0,0),2)
    cv.imshow('frame',contour_img)
    cv.imshow('mask o',mask_orange_img)
    cv.imshow('mask blue',mask_img)

    time.sleep(0.001)#prevents line array from filling too fast

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()